In [ ]:
# TEMPORIZADOR para comparar con el código viejo, comentar si no se usa.
t_inicio = cputime()


# ==============================================================================
# CÓDIGO DE REPLICACIÓN EXACTA (LOGICA ORIGINAL DE ORBITAS Y LISTAS)
# ==============================================================================
import sage.all
casos=[25]
def buscarEquivalencias(q):
    print(90*'-')
    print('INICIO DEL PROGRAMA (REPLICACIÓN EXACTA V2)')
    print(90*'-')
    print(f'INICIO DEL CASO Fq con q = {q}')

    k = GF(q, 'a')
    a = k.gen()
    print(f'Polinomio característico: {a.charpoly("y")}')

    # 1. Generación de elementos "a lo antiguo" para coincidir índices
    elem = []
    if is_prime(q):
        for i in k: elem.append(i)
    else:
        for i in range(q):
            if i == 0: elem.append(a*0)
            else: elem.append(a**i)
    
    menoselem = [-e for e in elem]

    # 2. Polinomios y Cuerpo
    R = PolynomialRing(k, 'x') # Usamos x para evitar conflicto
    x_poly = R.gen()
    monicirred2 = []
    for pol in R.polynomials(2):
        if pol.is_irreducible() and pol.leading_coefficient() == 1:
            monicirred2.append(pol)
    
    print(f'Hay {len(monicirred2)} polinomios mónicos irreducibles de grado 2')

    K = FunctionField(k, 'x')
    x = K.gen()
    O = K.maximal_order()
    
    # Usamos la lista nativa de places para coincidir con indices
    places_finite = K.places_finite()

    # 3. Matrices y Longitudes
    IMS = []
    matord = []
    for pmi in monicirred2:
        coeffs = pmi.list()
        a0, a1 = coeffs[0], coeffs[1]
        for t in k:
            B = matrix(k, 2, [t, -a0, 1, a1+t])
            if B.det() != 0:
                IMS.append(B)
                # Cálculo de orden (fuerza bruta como original)
                n = 1
                while True:
                    Bn = B**n
                    if Bn == Bn[0,0] * identity_matrix(k, 2):
                        matord.append(n)
                        break
                    n += 1

    NN = sorted(list(set(matord)))
    N = [n for n in NN if n < q+1 and n > 3 and (q+1) % n == 0]
    print(f'Las longitudes posibles para codigos sigma ciclicos son {N}')

    # -----------------------------------------------------------------------
    # NUCLEO DEL PROCESAMIENTO
    # -----------------------------------------------------------------------
    for n in N:
        print(30*'-')
        print(f'ESTUDIAREMOS AHORA CODIGOS DE LONGITUD {n} Y DIVISORES G DE GRADO 2')
        
        matn = [IMS[i] for i in range(len(IMS)) if matord[i] == n]
        print(30*'-')
        print(f'Hay {len(matn)} matrices de orden {n} sin reducir PGL2')

        MATGEN = [] # Lista acumulativa de matrices sistemáticas
        
        # Bucle principal de matrices
        for B in matn:
            # Matriz inversa (adjunta)
            Binv = matrix(k, 2, [B[1,1], -B[0,1], -B[1,0], B[0,0]])
            
            # Sigma
            try:
                sigma = K.hom([(B[0,0]*x + B[0,1]) / (B[1,0]*x + B[1,1])])
            except: continue

            # Lugares fijos (lg2fijos)
            lg2fijos = []
            for pol in monicirred2:
                im = sigma(pol)
                num = R(im.numerator())
                den = R(im.denominator())
                if num % pol == 0 and den % pol != 0:
                    lg2fijos.append(pol)
            
            # Si no hay lugares fijos, saltamos
            if not lg2fijos: continue

            # -----------------------------------------------------------
            # Generación de Órbitas (La lógica "rara" del original)
            # -----------------------------------------------------------
            orbitas = []
            for i in range(q):
                # Lógica original: buscar hache tal que menoselem[i] == elem[hache]
                # j se vuelve hache.
                val_buscado = menoselem[i]
                j = -1
                for idx, val in enumerate(elem):
                    if val == val_buscado:
                        j = idx
                        break
                
                # Verificamos si places_finite[j] ya está en alguna órbita
                P_start = places_finite[j]
                ya_esta = False
                for orb in orbitas:
                    if P_start in orb:
                        ya_esta = True
                        break
                
                if not ya_esta:
                    orbita = [P_start]
                    # aux comienza con a^i (que es elem[i] salvo i=0)
                    # OJO: elem[0] es 0. Si i=0, aux=[0].
                    val_aux = elem[i] 
                    aux=[a**i]#aux = [val_aux] 
                    
                    brk2 = 0
                    while brk2 == 0 and len(orbita) < n:
                        last_val = aux[-1]
                        num = Binv[0,0]*last_val + Binv[0,1]
                        deno = Binv[1,0]*last_val + Binv[1,1]
                        
                        if deno != 0:
                            new_val = num/deno
                            aux.append(new_val)
                            # Buscar índice k tal que menoselem[k] == new_val
                            # (Original: while new != menoselem[indice])
                            indice = -1
                            for k_idx, m_val in enumerate(menoselem):
                                if m_val == new_val:
                                    indice = k_idx
                                    break
                            
                            if indice != -1:
                                orbita.append(places_finite[indice])
                            else:
                                # Fallback raro del original
                                orbita.append(K.places()[0]) 
                                brk2 = 1
                        else:
                            orbita.append(K.places()[0])
                            brk2 = 1
                    
                    if len(orbita) == n:
                        orbitas.append(orbita)
            
            # -----------------------------------------------------------
            # Construcción de Códigos
            # -----------------------------------------------------------
            for Q in lg2fijos:
                I = O.ideal(Q)
                Base = I.divisor_of_zeros().basis_function_space()
                
                for orbita in orbitas:
                    # Evaluar
                    lista = []
                    for base in Base:
                        for p in orbita:
                            lista.append(base.evaluate(p))
                    
                    try:
                        # Crear matriz y código
                        matriz = matrix(k, len(Base), len(orbita), lista)
                        cod = LinearCode(matriz)
                        # Guardar LA SISTEMÁTICA (esto genera las duplicadas visuales)
                        MATGEN.append(cod.systematic_generator_matrix())
                    except:
                        pass

        # -----------------------------------------------------------------------
        # ANÁLISIS DE RESULTADOS (replicando flujo original)
        # -----------------------------------------------------------------------
        
        # 1. Eliminar duplicados exactos de matrices
        # En Sage las matrices son mutables, las convertimos a inmutables para usar set()
        Theta_set = set()
        Theta = []
        for M in MATGEN:
            M.set_immutable()
            if M not in Theta_set:
                Theta_set.add(M)
                Theta.append(M)
        
        theta_len = len(Theta)
        print(30*'-')
        print(f'HAY {theta_len} MATRICES GENERADORAS DE CODIGOS DISTINTOS, PERO QUIZAS EQUIVALENTES:')
        
        if theta_len > 0:
            # 2. Filtrado por Permutaciones (is_permutation_equivalent)
            # Queremos encontrar cuántas clases hay
            codigos_unicos = []
            matrices_representantes = []
            
            for mu in Theta:
                C1 = LinearCode(mu)
                es_nuevo = True
                for C2 in codigos_unicos:
                    if C1.is_permutation_equivalent(C2):
                        es_nuevo = False
                        break
                if es_nuevo:
                    codigos_unicos.append(C1)
                    matrices_representantes.append(mu)
            
            print(30*'-')
            print(f'Existen {len(codigos_unicos)} No equivalentes por permutaciones')
            ##################### TIEMPO De calculo para eqivalencias por permutaciones.
            print(f"Tiempo total de CPU: {cputime(t_inicio)} segundos, hasta llegar a este paso")
            print('Las matrices generadoras de estos codigos son:')
            for M in matrices_representantes:
                print(M)
                
            # 3. Filtrado Monomial (Tu "Debemos chequear...")
            print('Debemos chequear si son o no monomialmente equivalentes')
            
            # Re-verificar entre los representantes si son monomialmente equivalentes
            # (Tu código original hacía una lógica con diagonales aquí)
            
            from itertools import product
            unidades = [u for u in k if u != 0]
            
            finales = []
            for C_rep in codigos_unicos:
                es_nuevo_mono = True
                for C_final in finales:
                    # Check manual de monomial
                    # Probamos si C_rep ~ C_final * D
                    G_final = C_final.systematic_generator_matrix()
                    
                    is_equiv = False
                    # Check rápido permutación
                    if C_rep.is_permutation_equivalent(C_final):
                        is_equiv = True
                    else:
                        # Check diagonales
                        # Optimizacion: Primera entrada 1
                        for escalares in product(unidades, repeat=n-1):
                            diag = [k(1)] + list(escalares)
                            cols = [G_final.column(jj)*diag[jj] for jj in range(n)]
                            G_scaled = matrix(k, cols).transpose()
                            C_scaled = LinearCode(G_scaled)
                            if C_rep.is_permutation_equivalent(C_scaled):
                                is_equiv = True
                                break
                    
                    if is_equiv:
                        es_nuevo_mono = False
                        break
                
                if es_nuevo_mono:
                    finales.append(C_rep)

            print(30*'-')
            print(f'Existen {len(finales)} codigos sigma-ciclicos, salvo equivalencias,')
            print(f' de longitud {n} y dimension 3 sobre Fq con q = {q}')
            
            print(30*'-')
            print('Las matrices generadoras de codigos no equivalentes son')
            for C in finales:
                print(30*'-')
                print(C.systematic_generator_matrix())
            

    print(30*'-')
    print(f"Tiempo total de CPU: {cputime(t_inicio)} segundos para este q")
    print(f'FIN DEL CASO Fq con q = {q}')
    print(90*'-')
    print('FIN DEL PROGRAMA')
    print(90*'-')

for q in casos:
    buscarEquivalencias(q)
# TIEMPO FINAL, comentar si no se va a usar

In [ ]:
Nuevo con aux=[a**i].
Cuando el nuevo arrancó el caso q=25, el viejo aún estaba por q=9. 
cuando el viejo terminó el caso q=9, el nuevo estaba en el paso más costoso.
A la hora y un poco más, terminó el nuevo de hallar las matrices generadoras.
Caso aux=[aux_value] #supuestamente más rapido pero no sabemos si es seguro (correcto)
